In [11]:
import pandas as pd
import numpy as np
import warnings
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import callbacks
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

warnings.filterwarnings(action='ignore')

In [3]:
df = pd.read_csv("modified_train_0412.csv")
x_train = df.drop(columns=['loan_status', 'addr_state'])
y_train = df['loan_status']

In [4]:
nan_indices = np.isnan(x_train).any(axis=1)
x_train = x_train[~nan_indices]
y_train = y_train[~nan_indices]

In [5]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x_train)

# PCA로 차원 축소
pca = PCA(n_components=90)
X_pca = pca.fit_transform(X_scaled)
x_train = X_pca.astype('float32')
X_Train, X_Validation, Y_Train, Y_Validation = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)

In [7]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_train_over, y_train_over = sm.fit_resample(X_Train, Y_Train)
Y_Train=to_categorical(y_train_over, 2).astype(int)
Y_Validation=to_categorical(Y_Validation, 2).astype(int)

In [8]:
# Y_Train=to_categorical(Y_Train, 2).astype(int)
# Y_Validation=to_categorical(Y_Validation, 2).astype(int)

In [9]:
from keras import backend as K

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [13]:
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping

In [15]:
# model layers
model = keras.Sequential([
    Dense(45, input_dim=90,activation='relu'),
    Dropout(0,2),
    Dense(45,activation='relu'),
    Dropout(0.2),
    Dense(45, activation='relu'),
    Dense(2,activation='softmax'),
])

# optimizer
optimizer = Adam(learning_rate=0.01)

# compile
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy", f1_score, recall, precision])
# model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# train
epochs = 100
batch_size = 200000
early_stopping = EarlyStopping(min_delta=0.01)
model.fit(X_train_over, Y_Train, batch_size=batch_size, epochs=epochs)
# validation
# valid_loss, valid_acc = model.evaluate(X_Validation, Y_Validation, callbacks=[early_stopping])
valid_loss, valid_acc, valid_f1, valid_recall, valid_precision = model.evaluate(X_Validation, Y_Validation, callbacks=[early_stopping])
print("===================================")
print("Validation accuracy:", valid_acc)
print("Validation F1-score:", valid_f1)
print("Validation recall:", valid_recall)
print("Validation precesion:", valid_precision)

Epoch 1/100
7/7 [==============================] - 3s 260ms/step - loss: 0.7181 - accuracy: 0.5624 - f1_score: 0.5627 - recall: 0.5627 - precision: 0.5627
Epoch 2/100
7/7 [==============================] - 2s 232ms/step - loss: 0.6435 - accuracy: 0.6361 - f1_score: 0.6362 - recall: 0.6362 - precision: 0.6362
Epoch 3/100
7/7 [==============================] - 1s 208ms/step - loss: 0.6259 - accuracy: 0.6528 - f1_score: 0.6529 - recall: 0.6529 - precision: 0.6529
Epoch 4/100
7/7 [==============================] - 1s 193ms/step - loss: 0.6175 - accuracy: 0.6608 - f1_score: 0.6608 - recall: 0.6608 - precision: 0.6608
Epoch 5/100
7/7 [==============================] - 1s 201ms/step - loss: 0.6139 - accuracy: 0.6634 - f1_score: 0.6634 - recall: 0.6634 - precision: 0.6634
Epoch 6/100
7/7 [==============================] - 1s 202ms/step - loss: 0.6109 - accuracy: 0.6654 - f1_score: 0.6654 - recall: 0.6654 - precision: 0.6654
Epoch 7/100
7/7 [==============================] - 1s 194ms/step - los